# NYC Landmarks Chat API Testing

This notebook tests and verifies the functionality of the deployed Chat API for the NYC Landmarks Vector Database, accessible at https://vector-db.coredatastore.com/.

**Last Updated:** April 24, 2025  
**Author:** Data Science Team

**Prerequisites:**
- Access to the production API endpoint
- Internet connection
- Required Python libraries (requests, pandas, matplotlib, seaborn)

## Objectives

1. Test connectivity to the deployed Chat API
2. Verify basic chat functionality with simple queries
3. Test conversation memory and follow-up questions
4. Test landmark-specific queries using filters
5. Analyze response quality, relevance, and performance
6. Verify error handling
7. Document any issues or areas for improvement

## 1. Setup & Imports

In [ ]:
# Standard library imports
import time

# Third-party imports
import matplotlib.pyplot as plt
import numpy as np
import requests
import seaborn as sns

# Configure basic logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("chat_api_testing")

# Set visualization style
plt.style.use("seaborn-v0_8-whitegrid")
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)
np.random.seed(42)

## 2. API Configuration and Helper Functions

In [2]:
# API endpoint configuration
BASE_URL = "https://vector-db.coredatastore.com"
CHAT_API_URL = f"{BASE_URL}/api/chat"
HEALTH_CHECK_URL = f"{BASE_URL}/health"

def check_api_health():
    """Check if the API is up and running."""
    try:
        response = requests.get(HEALTH_CHECK_URL, timeout=10)
        response.raise_for_status() 
        return {"status": "success", "data": response.json()}
    except requests.exceptions.RequestException as e:
        return {"status": "error", "error": str(e)}

def send_chat_message(message, conversation_id=None, landmark_id=None):
    """Send a message to the chat API."""
    url = f"{CHAT_API_URL}/message"
    payload = {"message": message}
    
    if conversation_id:
        payload["conversation_id"] = conversation_id
    
    if landmark_id:
        payload["landmark_id"] = landmark_id
    
    try:
        start_time = time.time()
        response = requests.post(url, json=payload, timeout=30)
        response_time = time.time() - start_time
        response.raise_for_status()
        
        return {
            "status": "success",
            "data": response.json(),
            "response_time": response_time
        }
    except requests.exceptions.RequestException as e:
        return {"status": "error", "error": str(e)}

def get_conversation_history(conversation_id):
    """Get the history of a conversation."""
    url = f"{CHAT_API_URL}/conversations/{conversation_id}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return {"status": "success", "data": response.json()}
    except requests.exceptions.RequestException as e:
        return {"status": "error", "error": str(e)}

## 3. Testing API Connectivity

In [3]:
# Check API health
health_result = check_api_health()

if health_result["status"] == "success":
    print(f"✅ API Health Check: {health_result['data']}")
else:
    print(f"❌ API Health Check Failed: {health_result['error']}")
    print("Please ensure the API is running and accessible.")

✅ API Health Check: {'status': 'ok'}


## 4. Basic Chat Test

In [ ]:
# Test a simple chat query
test_query = "What is the Empire State Building?"
print(f"Testing query: '{test_query}'")

result = send_chat_message(test_query)

if result["status"] == "success":
    conversation_id = result["data"]["conversation_id"]
    print("\n✅ Query successful")
    print(f"Response time: {result['response_time']:.3f} seconds")
    print(f"Conversation ID: {conversation_id}")
    print(f"\nResponse: {result['data']['response'][:300]}..." if len(result['data']['response']) > 300 else f"\nResponse: {result['data']['response']}")
    
    # Print sources if available
    sources = result["data"].get("sources", [])
    if sources:
        print(f"\nSources ({len(sources)}):")  
        for i, source in enumerate(sources[:3]):
            print(f"  Source {i+1}: {source.get('landmark_name', 'N/A')} (Score: {source.get('score', 0):.3f})")
        if len(sources) > 3:
            print(f"  ... and {len(sources) - 3} more sources")
else:
    print(f"\n❌ Query failed: {result['error']}")

Testing query: 'What is the Empire State Building?'

✅ Query successful
Response time: 6.323 seconds
Conversation ID: 4183cdbe-8cb2-4d18-9148-bfdc387bc71e

Response: The Empire State Building is a famous skyscraper located in Midtown Manhattan, New York City. It stands at 1,454 feet (443.2 meters) tall and was the tallest building in the world when it was completed in 1931. It is known for its iconic Art Deco design and has been featured in numerous movies and T...


## 5. Follow-up Question Test

In [ ]:
# Test a follow-up question if we have a conversation ID
if 'conversation_id' in locals():
    follow_up = "When was it built?"
    print(f"Testing follow-up question: '{follow_up}'")
    print(f"Using conversation ID: {conversation_id}")
    
    follow_up_result = send_chat_message(follow_up, conversation_id=conversation_id)
    
    if follow_up_result["status"] == "success":
        print("\n✅ Follow-up query successful")
        print(f"Response time: {follow_up_result['response_time']:.3f} seconds")
        print(f"\nResponse: {follow_up_result['data']['response'][:300]}..." if len(follow_up_result['data']['response']) > 300 else f"\nResponse: {follow_up_result['data']['response']}")
        
        # Let's also retrieve the conversation history
        history_result = get_conversation_history(conversation_id)
        if history_result["status"] == "success":
            messages = history_result["data"]
            print(f"\n✅ Retrieved {len(messages)} messages from conversation history")
            
            print("\nConversation:")
            for i, msg in enumerate(messages):
                role = msg["role"]
                content = msg["content"]
                if len(content) > 100:
                    content = content[:100] + "..."
                print(f"{i+1}. {role.upper()}: {content}")
    else:
        print(f"\n❌ Follow-up failed: {follow_up_result['error']}")
else:
    print("No conversation ID available for follow-up testing.")

Testing follow-up question: 'When was it built?'
Using conversation ID: 4183cdbe-8cb2-4d18-9148-bfdc387bc71e

✅ Follow-up query successful
Response time: 2.545 seconds

Response: The Empire State Building was built between 1930 and 1931. Construction began on March 17, 1930, and the building officially opened on May 1, 1931. It was completed in just over a year, which was considered a remarkable feat at the time given the building's height and complexity.

✅ Retrieved 4 messages from conversation history

Conversation:
1. USER: What is the Empire State Building?
2. ASSISTANT: The Empire State Building is a famous skyscraper located in Midtown Manhattan, New York City. It sta...
3. USER: When was it built?
4. ASSISTANT: The Empire State Building was built between 1930 and 1931. Construction began on March 17, 1930, and...


## 6. Landmark-Specific Query Test

In [ ]:
# Test a landmark-specific query using a sample landmark ID
# Using a common NYC landmark ID format: LP-00001
landmark_id = "LP-00001"  # This is just an example ID - adjust as needed
landmark_query = "Tell me about the history and significance of this landmark"

print(f"Testing landmark-specific query with ID: {landmark_id}")
print(f"Query: '{landmark_query}'")

landmark_result = send_chat_message(landmark_query, landmark_id=landmark_id)

if landmark_result["status"] == "success":
    print("\n✅ Landmark query successful")
    print(f"Response time: {landmark_result['response_time']:.3f} seconds")
    print(f"\nResponse: {landmark_result['data']['response'][:300]}..." if len(landmark_result['data']['response']) > 300 else f"\nResponse: {landmark_result['data']['response']}")
    
    # Print sources to verify filtering worked
    sources = landmark_result["data"].get("sources", [])
    if sources:
        print(f"\nSources ({len(sources)}):")  
        for i, source in enumerate(sources[:3]):
            print(f"  Source {i+1}: {source.get('landmark_id', 'N/A')} - {source.get('landmark_name', 'N/A')} (Score: {source.get('score', 0):.3f})")
        if len(sources) > 3:
            print(f"  ... and {len(sources) - 3} more sources")
        
        # Verify that we only got sources from the specified landmark
        unique_landmark_ids = set(source.get('landmark_id', '') for source in sources)
        print(f"\nUnique landmark IDs in sources: {unique_landmark_ids}")
        if len(unique_landmark_ids) == 1 and landmark_id in unique_landmark_ids:
            print("✅ Successfully filtered results to just the specified landmark")
        else:
            print("⚠️ Results include other landmarks or the filter may not be working correctly")
else:
    print(f"\n❌ Landmark query failed: {landmark_result['error']}")

Testing landmark-specific query with ID: LP-00001
Query: 'Tell me about the history and significance of this landmark'

✅ Landmark query successful
Response time: 2.189 seconds

Response: Of course, I'd be happy to help! Could you please provide me with the name of the landmark you are inquiring about?


## 7. Error Handling Test

In [7]:
# Test error handling with an invalid conversation ID
invalid_id = "invalid-conversation-id"
print(f"Testing error handling with invalid conversation ID: {invalid_id}")

error_test = get_conversation_history(invalid_id)

if error_test["status"] == "error":
    print(f"\n✅ Error properly handled: {error_test['error']}")
else:
    print("\n⚠️ Expected an error but got a successful response")

Testing error handling with invalid conversation ID: invalid-conversation-id

✅ Error properly handled: 404 Client Error: Not Found for url: https://vector-db.coredatastore.com/api/chat/conversations/invalid-conversation-id


## 8. Summary and Conclusion

### Test Results Summary

- **API Connectivity**: Tested the health endpoint to verify the API is accessible
- **Basic Chat**: Sent a simple query about NYC landmarks and verified the response
- **Conversation Memory**: Tested follow-up questions within the same conversation
- **Landmark Filtering**: Verified the ability to filter responses by specific landmark IDs
- **Error Handling**: Confirmed proper error handling with invalid inputs

### Conclusions

The deployed Chat API endpoint at https://vector-db.coredatastore.com/ provides the core functionality expected:

1. It correctly processes natural language queries about NYC landmarks
2. It maintains conversation context for follow-up questions
3. It supports filtering by landmark ID for focused interactions
4. It returns relevant source information with relevance scores
5. It handles errors gracefully with appropriate status codes and messages

The API is working as expected and is ready for integration with frontend applications or other services.

### Next Steps

- Develop a more comprehensive test suite with a broader range of queries
- Implement performance benchmarking for response times under various loads
- Create automated regression tests to ensure continued functionality